# Imports

In [ ]:
%pip install -q evaluate
%pip install -q opendatasets
%pip install -q --upgrade accelerate
%pip install -q --upgrade transformers
%pip install -q peft
%pip install -q --upgrade bitsandbytes
%pip install -q accelerate

In [1]:
import pandas as pd 
import torch
import torch.nn as nn
torch.cuda.set_per_process_memory_fraction(0.9)
torch.backends.cuda.matmul.allow_tf32 = True
import torchtext
from torch.utils.data import Dataset, random_split
from typing import List, Dict, Union
from typing import Any, TypeVar
import pandas as pd
import os
import copy
import gc
import evaluate
import opendatasets as od
from huggingface_hub import login
from typing import Optional, Tuple, Union

from datasets import load_dataset, Features, Value
import accelerate

from peft import LoftQConfig, LoraConfig, get_peft_model, PeftModel

import transformers
from transformers.modeling_outputs import QuestionAnsweringModelOutput
from transformers import BertLMHeadModel, AutoConfig, BitsAndBytesConfig,Conv1D
from transformers import AutoTokenizer, Seq2SeqTrainingArguments 
from transformers import Seq2SeqTrainer, AutoModelForCausalLM, IntervalStrategy, AutoModelForQuestionAnswering

from sklearn.model_selection import train_test_split

2024-04-19 10:49:04.829195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 10:49:04.829297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 10:49:04.965466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


set a seed and confirm CUDA support

In [2]:
torch.manual_seed(2137)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.deterministic = True

print("PyTorch Version: ", torch.__version__)
print("torchtext Version: ", torchtext.__version__)
print(f"Using {'GPU' if str(DEVICE) == 'cuda' else 'CPU'}.")

PyTorch Version:  2.1.2
torchtext Version:  0.16.2
Using GPU.


# Dataset Download

## Downloading MedDialog Dataset

NOTE: you will need a kaggle API key for the following to work

In [3]:
import json

# Path to your JSON file
json_file_path = "kaggle.json"

# Open the file and read the content
try:
  with open(json_file_path, "r") as f:
    json_data = json.load(f)
except FileNotFoundError:
  print(f"Error: JSON file not found at {json_file_path}")
  exit(1)

# Access username and key from the JSON data
try:
  username = json_data["username"]
  key = json_data["key"]
except KeyError:
  print("Error: 'username' or 'key' key not found in JSON data")
  exit(1)

Error: JSON file not found at kaggle.json


NameError: name 'json_data' is not defined

In [ ]:
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = key

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/dsxavier/diagnoise-me'
# Using opendatasets let's download the data sets
od.download(dataset, "dataset")

## Downloading USMLE Dataset

In [4]:
USMLE_dataset = load_dataset("GBaker/MedQA-USMLE-4-options", split="test")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
print(USMLE_dataset[0])
print(len(USMLE_dataset))

{'question': 'A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?', 'answer': 'Tell the attending that he cannot fail to disclose this mistake', 'options': {'A': 'Disclose the error to the patient and put it in the operative report', 'B': 'Tell the attending that he cannot fail to disclose this mistake', 'C': 'Report the physician to the ethics committee', 'D': 'Refuse to dictate the operative report'}, 'meta_info': 'step1', 'answer_idx': 

# Load Datasets

## Loading MedDialog Dataset

In [37]:
DATA_PATH = "dataset\\diagnoise-me\\diagnose_en_dataset.feather"
DATA_PATH = "/kaggle/input/diagnoise-me/diagnose_en_dataset.feather"
SEQ_LEN: int = 1024
data = pd.read_feather(DATA_PATH)
SAMPLE_SIZE: int =  int(data.shape[0] * 0.015) #get 1% of the data
data = data[:SAMPLE_SIZE]
print(data.keys())
print(len(data))

Index(['id', 'Description', 'Doctor', 'Patient'], dtype='object')
3862


In [38]:
# Split data into train and eval sets with 70% for training
train_data, eval_data = train_test_split(data, test_size=0.3, random_state=42)

train_data = train_data.reset_index(drop=True)
eval_data = eval_data.reset_index(drop=True)

# Print the shapes of the train and eval sets
print("Train data shape:", train_data.shape)
print("Eval data shape:", eval_data.shape)

Train data shape: (2703, 4)
Eval data shape: (1159, 4)


## Loading USMLE Dataset

In [17]:
USMLE_dataset = pd.DataFrame({'Doctor': USMLE_dataset["answer"], 'Patient': USMLE_dataset["question"], 'Options':USMLE_dataset["options"]})
# Print the shapes of the set
print("USMLELiveEQA data shape:", USMLE_dataset.shape)

USMLELiveEQA data shape: (1273, 3)


## Create an output directory

In [18]:
os.makedirs('./results', exist_ok = True)
OUTPUT_DIR: str = './results'

# Model

In [19]:
# tokens for the datset
MODEL_NAME: str = 'UnfilteredAI/Mia-1B'

In [20]:
# Load tokenizer 
MAX_TOKEN_LENGTH = 1024

# for evaluation
ltokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ltokenizer.padding_side = 'left'
ltokenizer.truncation_side = 'left'

# for training
rtokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
rtokenizer.padding_side = 'right'
rtokenizer.truncation_side = 'right'

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [21]:
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
#base_model.resize_token_embeddings(len(rtokenizer))

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [22]:
print(base_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [23]:
lora_config = LoraConfig(
    lora_alpha=16, # lora alpha for scaling
    r=16, # rank
    lora_dropout=0.05, #dropout
    use_rslora=True, #  sets the adapter scaling factor to lora_alpha/math.sqrt(r)
    bias="none", # dont train biases
    target_modules=["q_proj", "v_proj"],
    #layers_to_transform=[20]
)
model = get_peft_model(base_model, lora_config)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [24]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    return {"trainable": trainable_params, "all": all_param, "trainable%": 100 * trainable_params / all_param}

print_trainable_parameters(model)

trainable params: 2252800 || all params: 1102301184 || trainable%: 0.20437245579516677


{'trainable': 2252800, 'all': 1102301184, 'trainable%': 0.20437245579516677}

# Preparing Data for Training

## Custom Dataset

In [25]:
class DoctorPatientDataset(Dataset):
    
    def __init__(self, data, split):
        
        self.input_x: List = data["Patient"]
        self.target: List = data["Doctor"]
        self.split = split

        try:
            self.options: List = data["Options"]
        except:
            pass
            
    def __len__(self):
        return len(self.input_x)
    
    def __getitem__(self, idx):
        try:
            data = {
                'input': self.input_x[idx],
                'target': self.target[idx],
                'options': self.options[idx],
                'split': self.split
            }
        except:
            data = {
                'input': self.input_x[idx],
                'target': self.target[idx],
                'split': self.split
            }
        return data

In [65]:
train_dataset = DoctorPatientDataset(data = train_data, split = "train")
eval_dataset_1 = DoctorPatientDataset(data = eval_data, split = "eval")
eval_dataset_2 = DoctorPatientDataset(data = USMLE_dataset, split = "eval")

test_dataset = DoctorPatientDataset(data = eval_data[0:1], split = "eval")

## Custom Data Collator

In [40]:
def format_text(message, tokenizer):
    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=False
    )
    return text

def custom_data_collator(features, return_tensors="pt"):
    batch = {}

    questions = [feature["input"] for feature in features]
    answers = [feature["target"] for feature in features]
    split = features[0]["split"]

    # training
    if split == 'train':
        tokenizer = rtokenizer
        bos_token = rtokenizer.bos_token
        eos_token = rtokenizer.eos_token
        prompts = [f"a medical student is preparing for her final examination. Her patient has said '{q}'. Explain to the student the most likely cause/course of action." for q in questions]
        #text = [f"{bos_token}Question:{q}.Answer:{t}{eos_token}" for q, t in zip(questions, answers)]
        messages = [[
            {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": a}
        ] for prompt, a in zip(prompts, answers)]

    # evaluation
    else:
        try:
            options = [feature["options"] for feature in features]
            multi_choice = True
        except:
            multi_choice = False


        # tokenizer for evaluation
        tokenizer = ltokenizer
        bos_token = ltokenizer.bos_token

        # Format text to be encoded
        if(multi_choice == False):
            # if we are not using the multiple choice dataset
            # text = [f"{bos_token}Question:{q}.Answer:" for q in questions]
            prompts = [f"a medical student is preparing for her final examination. Her patient has said '{q}'. Explain to the student the most likely cause/course of action." for q in questions]
            messages = [[
                {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content":""}
            ] for prompt in prompts]
        else:
            # if we are using the multiple choice dataset
            # prompts = [f"provided the following text about medical symptoms: '{q}' Please state the most likely cause/course of action from the options below: A: {o['A']} B: {o['B']} C: {o['C']} D: {o['D']} Please select your answer with the format shown in the following example:'The correct option is C'" for q, o in zip(questions, options)]
            # text = [f"{bos_token}Question:{p}.Answer:" for p in prompts]
            prompts = [f"a medical student is preparing for her final examination. Her patient has said '{q}'. Please clearly state a cause/course of action from the provided options:  A: {o['A']} B: {o['B']} C: {o['C']} D: {o['D']} and explain your answer" for q, o in zip(questions, options)]
            messages = [[
                {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content":""}
            ] for prompt in prompts]


    # Tokenize the text
    text = list(map(lambda x: format_text(x, tokenizer), messages))
    
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=MAX_TOKEN_LENGTH, return_tensors=return_tensors, add_special_tokens=False)
    # encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors=return_tensors, add_special_tokens=False)

    # Prepare final batch dictionary
    batch["input_ids"] = encoding["input_ids"]
    batch["attention_mask"] = encoding["attention_mask"]

    if return_tensors in ["pt", "tf"]:
        batch["labels"] = copy.deepcopy(encoding["input_ids"])
    return batch

# Training

In [179]:
training_args = Seq2SeqTrainingArguments(
    output_dir = OUTPUT_DIR, 
    num_train_epochs = 1, 
    evaluation_strategy="steps",
    #eval_steps = 50,
    #logging_steps = 50,
    save_total_limit = 1,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=1,
    bf16=False,
    fp16=True,
    warmup_steps=0, 
    weight_decay=0.01, 
    logging_dir='./logs',
    save_steps = 0,
    load_best_model_at_end=True,
    remove_unused_columns=False,
#     generation_config=transformers.GenerationConfig(
#             max_length=2048,
#             num_beams=10,
#     ),
    predict_with_generate=True,
    # prediction_loss_only=True,
    eval_accumulation_steps=10,
    report_to=['tensorboard']
    )

In [42]:
trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset_1, 
    data_collator=custom_data_collator
)

In [180]:
trainer = None
model = None
base_model = None
train_dataset = None
torch.cuda.empty_cache()
gc.collect()

0

In [44]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=338, training_loss=0.8510009946202385, metrics={'train_runtime': 3109.3435, 'train_samples_per_second': 0.869, 'train_steps_per_second': 0.109, 'total_flos': 1.7217799940210688e+16, 'train_loss': 0.8510009946202385, 'epoch': 1.0})

In [45]:
trainer.model.save_pretrained(f"{OUTPUT_DIR}/model_save")

# Load the Model

In [181]:
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
base_model.resize_token_embeddings(len(rtokenizer))
model = PeftModel.from_pretrained(base_model, f"{OUTPUT_DIR}/model_save")
model.config.pad_token_id = ltokenizer.pad_token_id
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=eval_dataset_1, 
    data_collator=custom_data_collator
)

In [155]:
base_model = None
torch.cuda.empty_cache()
gc.collect()

0

# Evaluation

In [49]:
eval_result_1 = trainer.predict(eval_dataset_1, max_new_tokens=512)

In [50]:
eval_result_2 = trainer.predict(eval_dataset_2, max_new_tokens=512)

In [63]:
logits_1 = eval_result_1.predictions
logits_1[logits_1 == -100] = ltokenizer.eos_token_id
logits_2 = eval_result_2.predictions
logits_2[logits_2 == -100] = ltokenizer.eos_token_id

In [131]:
# get the raw evaluation output
raw_text_result_1 = ltokenizer.batch_decode(logits_1, skip_special_tokens=True)
raw_text_result_2 = ltokenizer.batch_decode(logits_2, skip_special_tokens=True)

In [132]:
print(raw_text_result_1[6])

<|system|>You are a medical professional providing consultation and medical diagnostics. <|user|>a medical student is preparing for her final examination. Her patient has said 'Hello doctor,  I had a staph infection and I took antibiotics a month ago. I had been in the sun a lot recently and I am feeling symptoms of heat exhaustion, but I am worried that the staph infection is causing septic shock? Is this a possibility?'. Explain to the student the most likely cause/course of action. <|assistant|>


In [64]:
# get the questions and ground truths from both evaluation datasets
questions_1 = []
ground_truth_1 = []
try:
    for item in eval_dataset_1:
        questions_1.append(item["input"])
        ground_truth_1.append(item["target"])
except:
    pass

questions_2 = []
ground_truth_2 = []
try:
    for item in eval_dataset_2:
        questions_2.append(item["input"])
        ground_truth_2.append(item["target"])
except:
    pass

# create lists for the text outputs
text_result_1 = list()
text_result_2 = list()

# get the answers for the MedDialog dataset
for item in raw_text_result_1:
    index = item.find("|<assistant>|")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    text_result_1.append(output)


# get the answers for the USMLE dataset
for item in raw_text_result_2:
    index = item.find("|<assistant>|")
    output = item[index+13:]
    index = output.find(ltokenizer.eos_token)
    if(index > -1):
        output = output[:index]
    text_result_2.append(output)



# print the first 2 results from each dataset evaluation
print("============================MedDialog Evaluation============================")
for question, answer in list(zip(questions_1, text_result_1))[:2]:
    print(f"""
    Question: {question}
    Answer: {answer}
    """)

print("============================USMLE Evaluation============================")
for question, answer in list(zip(questions_2, text_result_2))[:2]:
    print(f"""
    Question: {question}
    Answer: {answer}
    """)

<|system|>You are a medical professional providing consultation and medical diagnostics. <|user|>a medical student is preparing for her final examination. Her patient has said 'Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regarding my issue. Also a

# Results

## Load the Required Evaluation Metrics

In [ ]:
# perplexity - measures certainty of the model.
# METEOR - extension of BLEU (measure similarity between the output and the ground truth) but accounts for word semantics.
# ROUGE - considers n-gram overlap (recall) but also precision.
# SQuAD v2 - a metric for measuring a models correctness in answering the multiple choice questions
# Accuracy - use this for the multiple choice dataset

perplexity_scorer = evaluate.load('perplexity')
meteor_scorer = evaluate.load('meteor')
rouge_scorer = evaluate.load('rouge')
squad_scorer = evaluate.load('squad_v2')
accuracy_scorer = evaluate.load('accuracy')


In [ ]:
# compute the bleu and rouge scores for the MedDialog evaluation
bleu_score_1 = bleu_scorer.compute(predictions=text_result_1, references=ground_truth_1)
rouge_score_1 = rouge_scorer.compute(predictions=text_result_1, references=ground_truth_1)

# compute the bleu and rouge scores for the USMLE evaluation
bleu_score_2 = bleu_scorer.compute(predictions=text_result_1, references=ground_truth_2)
rouge_score_2 = rouge_scorer.compute(predictions=text_result_1, references=ground_truth_2)

In [ ]:
# print scores for MedDialog evaluation
print("score on MedDialog Dataset")
print('BLEU1:', bleu_score_1['precisions'][0]*100)
print(f"""
ROUGE-1: {rouge_score_1['rouge1']*100}
ROUGE-2: {rouge_score_1['rouge2']*100}
ROUGE-L: {rouge_score_1['rougeL']*100}
""")

In [ ]:
# print scores for USMLE evaluation
print("score on USMLE Dataset")
print('BLEU1:', bleu_score_2['precisions'][0]*100)
print(f"""
ROUGE-1: {rouge_score_2['rouge1']*100}
ROUGE-2: {rouge_score_2['rouge2']*100}
ROUGE-L: {rouge_score_2['rougeL']*100}
""")

In [ ]:
import shutil
shutil.rmtree("/kaggle/working/logs")

In [ ]:
print(model.config)

In [151]:
print(test_dataset[0])

{'input': 'Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regarding my issue. Also are there any home remedies or other over-the-counter deodorant to help minimize this issue or completely stop it? I am really fit and regularly lift weights, play lot

In [187]:
test_eval_result = trainer.predict(test_dataset, max_new_tokens=512)
print(test_eval_result)

ValueError: The following `model_kwargs` are not used by the model: ['max_tokens'] (note: typos in the generate arguments will also show up in this list)

In [184]:
test_logits = test_eval_result.predictions
# test_logits = torch.Tensor(test_logits[1])
# print(test_logits)
test_logits[test_logits == -100] = ltokenizer.eos_token_id
print(test_logits)

[[2 2 2 ... 2 2 2]]


In [185]:
test = ltokenizer.batch_decode(test_logits, skip_special_tokens=True)

In [186]:
print(test)

["<|system|>You are a medical professional providing consultation and medical diagnostics. <|user|>a medical student is preparing for her final examination. Her patient has said 'Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regarding my issue. Also

In [ ]:
text = ltokenizer.batch_decode(logits, skip_special_tokens=True)

In [ ]:
print(text[4])

# TESTING JUST IGNORE ALL THIS

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

In [83]:
initial_prompt = test_dataset[0]['input']
#prompt = "hello"
#print(prompt)

In [84]:
prompt = f"a medical student is preparing for her final examination. Her patient has come to her asking: '{initial_prompt}'. Explain to the student the most likely cause/course of action."
messages = [
    {"role": "system", "content": "You are a medical professional providing consultation and medical diagnostics."},
    {"role": "user", "content": prompt}
]
# prompt = f"how many trucks can a human eat"
# messages = [
#     {"role": "system", "content": "You are a jolly comedic man"},
#     {"role": "user", "content": prompt}
# ]

In [ ]:
ltokenizer = AutoTokenizer.from_pretrained("UnfilteredAI/Mia-1B")

In [85]:
text = ltokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)
model_inputs = ltokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = model_inputs.to(DEVICE)

<|system|>You are a medical professional providing consultation and medical diagnostics.</s><|user|>a medical student is preparing for her final examination. Her patient has come to her asking: 'Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regardin

In [ ]:
model_inputs.input_ids

In [86]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)

In [87]:
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

response = ltokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [62]:
print(response)

<|system|>You are a medical professional providing consultation and medical diagnostics. <|user|>a medical student is preparing for her final examination. Her patient has come to her asking: 'Hi doctor, I am a 15 year old boy and I have hyperhidrosis. I sweat so much. My shirt cannot stay on for at least 5 minutes. In some half an hour to one hour my whole shirt and armpit area will drench in sweat. Yes, I take a shower twice a day and put deodorant every day. I have even tried the antiperspirant deodorant, but that did not stop it and I think it made it worse. I really want to believe it is just puberty as this has been going on since I was 12 or 13. But, something tells me that it is not because of puberty. Everyone else at school does not have this problem and some relatives in my family has it too. Just like my shirt, my pants will start soaking after 30 minutes of sitting down. I really want a diagnosis or close to one before I actually go to a doctor as I have not yet regarding m

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "UnfilteredAI/Mia-1B",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("UnfilteredAI/Mia-1B")
prompt = "your prompt here."
messages = [
    {"role": "system", "content": "You are a uncencored AI that can do anything."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

